In [138]:
import random
from deap import base, creator, tools, algorithms
import pandas as pd
import csv
from collections import defaultdict

In [139]:
# PARÁMETROS
# los deduce de los slots

# depende si quiero usar los slots de bloques de hora y media o los de 45 minutos
#df_slots = pd.read_csv("datos/slots.csv")
df_slots = pd.read_csv("../datos/slots_divididos_dic_2024.csv")

DIAS = df_slots['dia'].unique().tolist()
SLOTS = df_slots['slot'].unique().tolist()

print(DIAS)
print(SLOTS)

W1, W2, W3 = 10, 3, 0  # pesos del costo (días, disponibilidad 2, huecos)

['Lunes', 'Martes', 'Miércoles', 'Jueves']
[1, 2, 3, 4, 5, 6, 7, 8]


In [140]:
def cargar_asignaciones(path):
    asign = {}
    with open(path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            asign[row['alumno']] = [row['jurado1'], row['jurado2'], row['jurado3']]
    return asign

def cargar_disponibilidad(path):
    disp = defaultdict(lambda: defaultdict(dict))
    with open(path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            jur = row['jurado']
            dia = row['dia']
            slot = int(row['slot'])

            #ignoro el "si, si fuera necesario"
            if (row['disponibilidad']==1):
                row['disponibilidad']=2
                
            disp[jur][dia][slot] = int(row['disponibilidad'])
    return disp

In [141]:
asignaciones = cargar_asignaciones("../datos/asignaciones_dic_2024.csv")
disponibilidad = cargar_disponibilidad("../datos/disponibilidad_dic_2024.csv")

In [142]:
asignaciones

{'Sarmiento': ['jose840602@gmail.com',
  'rodrigo.cardenas.sz@gmail.com',
  'farfan.roberto.f@gmail.com'],
 'Fernandez': ['adrianchamudis@gmail.com',
  'ingalbertopacheco@gmail.com',
  'japsiete@gmail.com'],
 'Villarraza': ['lmarianocampos@gmail.com',
  'emilianorm@gmail.com',
  'rraulemilioromero@gmail.com'],
 'Alianak': ['farfan.roberto.f@gmail.com',
  'gerardox2000@gmail.com',
  'denisjorgegenero@gmail.com'],
 'Valdez': ['facundolucianna@gmail.com',
  'maxit1992@gmail.com',
  'rodolfopaganini@gmail.com'],
 'Acerbo': ['bureuclara@gmail.com',
  'maxit1992@gmail.com',
  'rodolfopaganini@gmail.com'],
 'Baffo': ['farfan.roberto.f@gmail.com',
  'cyanez@fi.uba.ar',
  'edgar.torre.acad.2019@gmail.com'],
 'Bampini Basualdo': ['roberto.axt@gmail.com',
  'javifanelli@gmail.com',
  'gabrielcarutti@gmail.com'],
 'Carreño': ['pgomez@fi.uba.ar ',
  'eduardo.filomena@uner.edu.ar',
  'jcruz@fi.uba.ar'],
 'Vásquez González': ['denisjorgegenero@gmail.com',
  'apermingeat@gmail.com',
  'dessaya@gmail.c

In [143]:
# ==============================
# VALIDACIÓN DE DATOS
# ==============================

def validar_datos(asignaciones, disponibilidad, DIAS, SLOTS):
    """
    Valida que los datos estén correctamente estructurados antes de ejecutar el GA
    """
    print("\n" + "="*70)
    print("VALIDANDO DATOS DE ENTRADA")
    print("="*70)
    
    errores = []
    warnings = []
    
    # 1. Verificar estructura de asignaciones
    print(f"\n✓ Total alumnos: {len(asignaciones)}")
    for alumno, jurados in list(asignaciones.items())[:3]:
        print(f"  - {alumno}: {len(jurados)} jurados")
    
    # 2. Verificar que todos los jurados en asignaciones existen en disponibilidad
    todos_jurados = set()
    jurados_disponibles = set(disponibilidad.keys())
    
    print(f"\n✓ Total jurados únicos: {len(todos_jurados)}")
    print(f"✓ Jurados en disponibilidad: {len(disponibilidad)}")
    
    # Verificar cada jurado
    for alumno, jurados in asignaciones.items():
        for j in jurados:
            todos_jurados.add(j)
            # Use 'in jurados_disponibles' instead of 'in disponibilidad' 
            # to avoid defaultdict auto-creation issues
            if j not in jurados_disponibles:
                errores.append(f"Jurado '{j}' (alumno {alumno}) NO existe en disponibilidad")
    
    print(f"\n✓ Total jurados únicos en asignaciones: {len(todos_jurados)}")
    print(f"✓ Jurados en disponibilidad: {len(jurados_disponibles)}")
    
    # 3. Verificar estructura de disponibilidad
    if disponibilidad:
        primer_jurado = list(disponibilidad.keys())[0]
        dias_en_data = list(disponibilidad[primer_jurado].keys())
        print(f"\n✓ Días en disponibilidad: {dias_en_data}")
        print(f"✓ DIAS configurado: {DIAS}")
        
        # Verificar que DIAS coincida
        for dia in DIAS:
            if dia not in dias_en_data:
                errores.append(f"Día '{dia}' en DIAS no existe en disponibilidad")
        
        # Verificar slots
        if dias_en_data:
            slots_en_data = list(disponibilidad[primer_jurado][dias_en_data[0]].keys())
            print(f"✓ Slots en disponibilidad: {slots_en_data}")
            print(f"✓ SLOTS configurado: {SLOTS}")
            
            for slot in SLOTS:
                if slot not in slots_en_data:
                    warnings.append(f"Slot {slot} en SLOTS no existe en disponibilidad del primer jurado")
    
    # 4. Verificar factibilidad básica
    print(f"\n✓ Slots disponibles totales: {len(DIAS) * len(SLOTS)}")
    print(f"✓ Sesiones a programar: {len(asignaciones)}")
    
    if len(asignaciones) > len(DIAS) * len(SLOTS):
        errores.append(f"¡IMPOSIBLE! Necesitas {len(asignaciones)} slots pero solo hay {len(DIAS) * len(SLOTS)}")
    
    # 5. Verificar disponibilidad real
    print("\n✓ Verificando disponibilidad de cada alumno...")
    alumnos_sin_slots = []
    for alumno, jurados in asignaciones.items():
        slots_validos = 0
        for dia in DIAS:
            for slot in SLOTS:
                # Verificar si TODOS los jurados están disponibles
                todos_ok = True
                for j in jurados:
                    if j not in disponibilidad:
                        todos_ok = False
                        break
                    if dia not in disponibilidad[j]:
                        todos_ok = False
                        break
                    val = disponibilidad[j][dia].get(slot, 0)
                    if val == 0:
                        todos_ok = False
                        break
                
                if todos_ok:
                    slots_validos += 1
        
        if slots_validos == 0:
            alumnos_sin_slots.append(alumno)
    
    if alumnos_sin_slots:
        print(f"\n⚠️  ALUMNOS SIN NINGÚN SLOT VÁLIDO: {len(alumnos_sin_slots)}")
        for alumno in alumnos_sin_slots[:5]:
            print(f"  - {alumno}")
        if len(alumnos_sin_slots) > 5:
            print(f"  ... y {len(alumnos_sin_slots) - 5} más")
    else:
        print(f"✓ Todos los alumnos tienen al menos un slot válido")
    
    # Mostrar resumen
    print("\n" + "="*70)
    if errores:
        print("❌ ERRORES CRÍTICOS ENCONTRADOS:")
        for err in errores:
            print(f"  - {err}")
        return False
    elif warnings:
        print("⚠️  WARNINGS:")
        for warn in warnings:
            print(f"  - {warn}")
        print("\n✓ Puedes continuar pero revisa los warnings")
        return True
    else:
        print("✅ VALIDACIÓN EXITOSA - Datos correctos")
        return True




In [144]:
# ==============================
# CONFIGURACIÓN DE DEAP
# ==============================

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", dict, fitness=creator.FitnessMax)

/Users/ari/opt/miniconda3/envs/ae-env/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/ari/opt/miniconda3/envs/ae-env/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [145]:
# ==============================
# FUNCIÓN DE FITNESS
# ==============================

def calcular_fitness(individuo, asignaciones, disponibilidad, W1=1000, W2=10, W3=100):
    """
    Convierte el costo en fitness (negativo para maximizar = minimizar costo)
    """
    # Verificar que todos los alumnos estén asignados
    if len(individuo) != len(asignaciones):
        return (-1e9,)  # Penalización masiva por solución incompleta
    
    uso_dispon2 = 0
    huecos = 0
    penalizacion = 0

    # R1: Jurados disponibles (HARD CONSTRAINT - debe ser 1 o 2, nunca 0)
    for alumno, (dia, slot) in individuo.items():
        if alumno not in asignaciones:
            return (-1e9,)
        
        jurados = asignaciones[alumno]
        for jur in jurados:
            # Verificar que el jurado existe y el día existe
            if jur not in disponibilidad:
                return (-1e9,)
            if dia not in disponibilidad[jur]:
                return (-1e9,)
            
            val = disponibilidad[jur][dia].get(slot, 0)
            
            # Si cualquier jurado NO está disponible (val == 0), penalización masiva
            if val == 0:
                return (-1e9,)
            
            # Si usa disponibilidad secundaria (val == 2), penalizar levemente
            if val == 2:
                uso_dispon2 += 1

    # R2: Detectar solapamientos (dos alumnos en mismo slot) - HARD CONSTRAINT
    slot_counts = {}
    for alumno, (dia, slot) in individuo.items():
        key = (dia, slot)
        slot_counts[key] = slot_counts.get(key, 0) + 1
        if slot_counts[key] > 1:
            # Solapamiento detectado - penalización masiva
            penalizacion += 10000 * slot_counts[key]

    # R3: Detectar huecos dentro de cada día (soft constraint)
    for dia in DIAS:
        slots_ocupados = sorted([s for d, s in individuo.values() if d == dia])
        if len(slots_ocupados) > 1:
            for a, b in zip(slots_ocupados[:-1], slots_ocupados[1:]):
                if b - a > 1:
                    huecos += b - a - 1

    # R4: Calcular días usados
    dias_usados = len(set(d for d, s in individuo.values()))

    # Calcular costo total
    costo = W1 * dias_usados + W2 * uso_dispon2 + W3 * huecos + penalizacion
    
    # Retornar fitness negativo (para maximizar = minimizar costo)
    return (-costo,)

In [146]:
# ==============================
# CREACIÓN DE INDIVIDUOS
# ==============================

def crear_individuo_heuristico(asignaciones, disponibilidad, DIAS, SLOTS):
    """
    Crea un individuo usando heurística greedy: 
    asignar cada alumno al primer slot disponible donde todos sus jurados estén libres
    Y SIN SOLAPAMIENTOS
    """
    individuo = {}
    slots_usados = set()  # Para evitar solapamientos
    
    alumnos = list(asignaciones.keys())
    random.shuffle(alumnos)  # orden aleatorio para diversidad
    
    for alumno in alumnos:
        asignado = False
        jurados = asignaciones[alumno]
        
        # Intentar días y slots en orden aleatorio
        dias_slots = [(d, s) for d in DIAS for s in SLOTS]
        random.shuffle(dias_slots)
        
        for dia, slot in dias_slots:
            slot_key = (dia, slot)
            
            # CRÍTICO: Verificar que el slot NO esté usado (evitar solapamiento)
            if slot_key in slots_usados:
                continue
            
            # Verificar que TODOS los jurados estén disponibles (val != 0)
            todos_disponibles = True
            for j in jurados:
                if j not in disponibilidad:
                    todos_disponibles = False
                    break
                if dia not in disponibilidad[j]:
                    todos_disponibles = False
                    break
                val = disponibilidad[j][dia].get(slot, 0)
                if val == 0:  # Jurado NO disponible
                    todos_disponibles = False
                    break
            
            if todos_disponibles:
                individuo[alumno] = (dia, slot)
                slots_usados.add(slot_key)
                asignado = True
                break
        
        if not asignado:
            # Si no se pudo asignar con constraints, buscar cualquier slot libre
            # (será penalizado pero al menos no se solapa)
            for dia, slot in dias_slots:
                slot_key = (dia, slot)
                if slot_key not in slots_usados:
                    individuo[alumno] = (dia, slot)
                    slots_usados.add(slot_key)
                    asignado = True
                    break
            
            # Si aún no se asignó, forzar en slot aleatorio (muy malo pero completo)
            if not asignado:
                dia = random.choice(DIAS)
                slot = random.choice(SLOTS)
                individuo[alumno] = (dia, slot)
    
    return creator.Individual(individuo)


def crear_individuo_aleatorio(asignaciones, DIAS, SLOTS):
    """
    Crea un individuo completamente aleatorio
    """
    individuo = {}
    for alumno in asignaciones.keys():
        dia = random.choice(DIAS)
        slot = random.choice(SLOTS)
        individuo[alumno] = (dia, slot)
    return creator.Individual(individuo)

In [147]:
# ==============================
# OPERADORES GENÉTICOS
# ==============================

def mutar_individuo(individuo, asignaciones, disponibilidad, DIAS, SLOTS, indpb=0.2):
    """
    Muta un individuo cambiando la asignación de algunos alumnos
    RESPETANDO: no solapamientos y disponibilidad de jurados
    """
    alumnos = list(individuo.keys())
    
    for alumno in alumnos:
        if random.random() < indpb:
            jurados = asignaciones[alumno]
            dia_actual, slot_actual = individuo[alumno]
            
            # Buscar slots válidos (no usados por otros y con jurados disponibles)
            opciones_validas = []
            for dia in DIAS:
                for slot in SLOTS:
                    slot_key = (dia, slot)
                    
                    # Verificar que no esté usado por OTRO alumno
                    usado_por_otro = False
                    for otro_alumno, otro_slot in individuo.items():
                        if otro_alumno != alumno and otro_slot == slot_key:
                            usado_por_otro = True
                            break
                    
                    if usado_por_otro:
                        continue
                    
                    # Verificar disponibilidad de TODOS los jurados
                    todos_disponibles = True
                    for j in jurados:
                        if j not in disponibilidad or dia not in disponibilidad[j]:
                            todos_disponibles = False
                            break
                        val = disponibilidad[j][dia].get(slot, 0)
                        if val == 0:
                            todos_disponibles = False
                            break
                    
                    if todos_disponibles:
                        opciones_validas.append(slot_key)
            
            # Si hay opciones válidas, elegir una
            if opciones_validas:
                individuo[alumno] = random.choice(opciones_validas)
            # Si no hay opciones válidas, dejar como está (no empeorar)
    
    return individuo,


def crossover_individuos(ind1, ind2):
    """
    Cruce de dos puntos: intercambia segmentos de alumnos entre dos padres
    CUIDADO: Puede crear solapamientos que serán reparados después
    """
    alumnos = list(ind1.keys())
    if len(alumnos) < 2:
        return ind1, ind2
    
    # Seleccionar dos puntos de corte
    size = len(alumnos)
    pt1 = random.randint(1, size - 1)
    pt2 = random.randint(1, size - 1)
    if pt1 > pt2:
        pt1, pt2 = pt2, pt1
    
    # Intercambiar segmento
    alumnos_segmento = alumnos[pt1:pt2]
    for alumno in alumnos_segmento:
        ind1[alumno], ind2[alumno] = ind2[alumno], ind1[alumno]
    
    # Reparar solapamientos en ambos individuos
    ind1 = reparar_solapamientos_simple(ind1, DIAS, SLOTS)
    ind2 = reparar_solapamientos_simple(ind2, DIAS, SLOTS)
    
    return ind1, ind2


def reparar_solapamientos_simple(individuo, DIAS, SLOTS):
    """
    Repara solapamientos moviendo alumnos duplicados a slots libres
    """
    slot_a_alumno = defaultdict(list)
    for alumno, slot_tuple in individuo.items():
        slot_a_alumno[slot_tuple].append(alumno)
    
    # Encontrar y resolver conflictos
    for slot_tuple, alumnos_list in slot_a_alumno.items():
        if len(alumnos_list) > 1:
            # Mantener el primero, mover los demás
            for alumno in alumnos_list[1:]:
                # Buscar cualquier slot libre
                slots_usados = set(individuo.values())
                for dia in DIAS:
                    for slot in SLOTS:
                        slot_key = (dia, slot)
                        if slot_key not in slots_usados:
                            individuo[alumno] = slot_key
                            slots_usados.add(slot_key)
                            break
                    else:
                        continue
                    break
    
    return individuo


def reparar_individuo(individuo, asignaciones, disponibilidad, DIAS, SLOTS):
    """
    Intenta reparar un individuo eliminando solapamientos y violaciones de constraints
    """
    # Detectar solapamientos
    slot_a_alumno = defaultdict(list)
    for alumno, slot_tuple in individuo.items():
        slot_a_alumno[slot_tuple].append(alumno)
    
    # Resolver solapamientos moviendo alumnos
    for slot_tuple, alumnos_list in slot_a_alumno.items():
        if len(alumnos_list) > 1:
            # Mantener el primero, mover los demás
            for alumno in alumnos_list[1:]:
                jurados = asignaciones[alumno]
                # Buscar nuevo slot válido
                for dia in DIAS:
                    for slot in SLOTS:
                        if (dia, slot) not in individuo.values():
                            if all(disponibilidad[j][dia].get(slot, 0) != 0 for j in jurados):
                                individuo[alumno] = (dia, slot)
                                break
    
    return individuo,

In [148]:
# ==============================
# CONFIGURACIÓN DEL TOOLBOX
# ==============================

def setup_ga(asignaciones, disponibilidad, DIAS, SLOTS, W1=1000, W2=10, W3=100):
    """
    Configura el toolbox de DEAP para el problema
    """
    toolbox = base.Toolbox()
    
    # Registrar funciones con parámetros fijos
    toolbox.register("individual_heuristic", 
                     crear_individuo_heuristico, 
                     asignaciones, disponibilidad, DIAS, SLOTS)
    
    toolbox.register("individual_random",
                     crear_individuo_aleatorio,
                     asignaciones, DIAS, SLOTS)
    
    def init_population(n):
        # 70% heurísticos, 30% aleatorios
        pop = []
        for i in range(n):
            if i < int(0.7 * n):
                pop.append(toolbox.individual_heuristic())
            else:
                pop.append(toolbox.individual_random())
        return pop
    
    toolbox.register("population", init_population)
    
    toolbox.register("evaluate", 
                     calcular_fitness, 
                     asignaciones=asignaciones, 
                     disponibilidad=disponibilidad,
                     W1=W1, W2=W2, W3=W3)
    
    toolbox.register("mate", crossover_individuos)
    
    toolbox.register("mutate", 
                     mutar_individuo, 
                     asignaciones=asignaciones,
                     disponibilidad=disponibilidad,
                     DIAS=DIAS,
                     SLOTS=SLOTS,
                     indpb=0.2)
    
    toolbox.register("select", tools.selTournament, tournsize=5)
    
    toolbox.register("repair", 
                     reparar_individuo,
                     asignaciones=asignaciones,
                     disponibilidad=disponibilidad,
                     DIAS=DIAS,
                     SLOTS=SLOTS)
    
    return toolbox

In [149]:
# ==============================
# EJECUCIÓN DEL ALGORITMO GENÉTICO
# ==============================

def ejecutar_ga(asignaciones, disponibilidad, DIAS, SLOTS, W1=1000, W2=10, W3=100,
                pop_size=200, ngen=150, cxpb=0.7, mutpb=0.3):
    """
    Ejecuta el algoritmo genético
    """
    random.seed(42)
    
    toolbox = setup_ga(asignaciones, disponibilidad, DIAS, SLOTS, W1, W2, W3)
    
    # Crear población inicial
    pop = toolbox.population(n=pop_size)
    
    # Hall of Fame para guardar mejores soluciones
    hof = tools.HallOfFame(3)
    
    # Estadísticas
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: sum(v[0] for v in x) / len(x))
    stats.register("min", lambda x: min(v[0] for v in x))
    stats.register("max", lambda x: max(v[0] for v in x))
    
    print(f"Total de alumnos a programar: {len(asignaciones)}")
    print("Iniciando algoritmo genético...\n")
    
    # Ejecutar GA
    algorithms.eaSimple(
        pop, toolbox,
        cxpb=cxpb,
        mutpb=mutpb,
        ngen=ngen,
        stats=stats,
        halloffame=hof,
        verbose=True
    )
    
    return hof, pop

In [150]:
# Check those specific jurors
jurados_problema = ['pgomez@fi.uba.ar', 'maabruno@fi.uba.ar', 'macroldan@fi.uba.ar']

print("Verificación manual:")
for j in jurados_problema:
    existe = j in disponibilidad
    print(f"  {j}: existe={existe}")
    
    # Try with strip
    existe_stripped = j.strip() in disponibilidad
    print(f"    con strip: {existe_stripped}")
    
    # Check all jurors that contain part of this email
    matching = [k for k in disponibilidad.keys() if 'pgomez' in k or 'maabruno' in k or 'macroldan' in k]
    if matching:
        print(f"    Encontrados similares: {matching}")

print(f"\nPrimeros 5 jurados en disponibilidad:")
for i, j in enumerate(list(disponibilidad.keys())[:5]):
    print(f"  '{j}' (repr: {repr(j)})")

Verificación manual:
  pgomez@fi.uba.ar: existe=True
    con strip: True
    Encontrados similares: ['maabruno@fi.uba.ar', 'macroldan@fi.uba.ar', 'pgomez@fi.uba.ar']
  maabruno@fi.uba.ar: existe=True
    con strip: True
    Encontrados similares: ['maabruno@fi.uba.ar', 'macroldan@fi.uba.ar', 'pgomez@fi.uba.ar']
  macroldan@fi.uba.ar: existe=True
    con strip: True
    Encontrados similares: ['maabruno@fi.uba.ar', 'macroldan@fi.uba.ar', 'pgomez@fi.uba.ar']

Primeros 5 jurados en disponibilidad:
  'Jose.Alamos@haw-hamburg.de' (repr: 'Jose.Alamos@haw-hamburg.de')
  'adrianchamudis@gmail.com' (repr: 'adrianchamudis@gmail.com')
  'alicialmon@gmail.com' (repr: 'alicialmon@gmail.com')
  'apermingeat@gmail.com' (repr: 'apermingeat@gmail.com')
  'avirgillo.95@gmail.com' (repr: 'avirgillo.95@gmail.com')


In [151]:
# ==============================
# VISUALIZACIÓN DE RESULTADOS
# ==============================

def mostrar_solucion(solucion, asignaciones, disponibilidad, idx=1, verbose=True):
    """
    Muestra una solución en formato legible con validación detallada
    """
    print("\n" + "="*70)
    print(f" SOLUCIÓN {idx} (Fitness: {solucion.fitness.values[0]:.2f})")
    print("="*70)
    
    # Calcular métricas
    dias_usados = len(set(d for d, s in solucion.values()))
    alumnos_asignados = len(solucion)
    
    print(f"\nAlumnos asignados: {alumnos_asignados}/{len(asignaciones)}")
    print(f"Días utilizados: {dias_usados}")
    
    # VALIDACIÓN: Detectar problemas
    errores_disponibilidad = []
    errores_solapamiento = []
    
    # Verificar solapamientos
    slot_counts = defaultdict(int)
    for alumno, (dia, slot) in solucion.items():
        slot_counts[(dia, slot)] += 1
    
    for slot_key, count in slot_counts.items():
        if count > 1:
            errores_solapamiento.append(f"{slot_key}: {count} alumnos")
    
    # Verificar disponibilidad
    for alumno, (dia, slot) in solucion.items():
        jurados = asignaciones.get(alumno, [])
        for j in jurados:
            if j not in disponibilidad:
                errores_disponibilidad.append(f"{alumno} - Jurado {j} no existe en disponibilidad")
                continue
            if dia not in disponibilidad[j]:
                errores_disponibilidad.append(f"{alumno} - Jurado {j} no tiene info para {dia}")
                continue
            
            val = disponibilidad[j][dia].get(slot, 0)
            if val == 0:
                errores_disponibilidad.append(
                    f"{alumno} → {dia} Slot {slot}: Jurado {j} NO disponible (val={val})"
                )
    
    # Mostrar errores encontrados
    if errores_solapamiento:
        print("\n⚠️  ERRORES DE SOLAPAMIENTO:")
        for err in errores_solapamiento:
            print(f"  - {err}")
    
    if errores_disponibilidad:
        print("\n❌ ERRORES DE DISPONIBILIDAD:")
        for err in errores_disponibilidad[:10]:  # Mostrar solo primeros 10
            print(f"  - {err}")
        if len(errores_disponibilidad) > 10:
            print(f"  ... y {len(errores_disponibilidad) - 10} más")
    
    if not errores_disponibilidad and not errores_solapamiento:
        print("\n✅ SOLUCIÓN VÁLIDA - Sin conflictos detectados")
    
    # Agrupar por día
    por_dia = defaultdict(list)
    for alumno, (dia, slot) in solucion.items():
        jurados = asignaciones[alumno]
        por_dia[dia].append((slot, alumno, jurados))
    
    # Mostrar por día
    if verbose:
        for dia in DIAS:
            if dia in por_dia:
                print(f"\n {dia}:")
                for slot, alumno, jurados in sorted(por_dia[dia]):
                    # Verificar disponibilidad
                    dispon_str = []
                    for j in jurados:
                        val = disponibilidad[j][dia].get(slot, 0)
                        if val == 0:
                            dispon_str.append(f"{j}❌(val={val})")
                        elif val == 2:
                            dispon_str.append(f"{j}⚠️(val={val})")
                        else:
                            dispon_str.append(f"{j}✓(val={val})")
                    
                    print(f"  Slot {slot:2d} | {alumno:15s} | {', '.join(dispon_str)}")


def crear_dataframe_resultado(solucion, asignaciones, df_slots=None):
    """
    Convierte la solución en un DataFrame para exportar
    """
    filas = []
    for alumno, (dia, slot) in solucion.items():
        jurados = ', '.join(asignaciones.get(alumno, []))
        filas.append({
            'Alumno': alumno,
            'Día': dia,
            'Slot': slot,
            'Jurados': jurados
        })
    
    df = pd.DataFrame(filas)
    
    # Ordenar por día y slot
    dias_orden = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
    df['Día'] = pd.Categorical(df['Día'], categories=dias_orden, ordered=True)
    df = df.sort_values(['Día', 'Slot'])
    
    # Merge con horarios si existe df_slots
    if df_slots is not None:
        df = df.merge(
            df_slots[['dia', 'slot', 'rango_horario']],
            left_on=['Día', 'Slot'],
            right_on=['dia', 'slot'],
            how='left'
        )
        df = df.drop(columns=['dia', 'slot'])
        df = df[['Día', 'rango_horario', 'Alumno', 'Jurados']]
    
    return df

In [152]:
# ==============================
# EJEMPLO DE USO
# ==============================

# ==============================
# FUNCIONES DE EJECUCIÓN SEGURA
# ==============================

def ejecutar_validacion(asignaciones, disponibilidad, DIAS, SLOTS):
    """Ejecuta solo la validación"""
    try:
        resultado = validar_datos(asignaciones, disponibilidad, DIAS, SLOTS)
        return resultado
    except Exception as e:
        print(f"❌ Error durante validación: {e}")
        import traceback
        traceback.print_exc()
        return False


def ejecutar_ga_seguro(asignaciones, disponibilidad, DIAS, SLOTS, W1, W2, W3):
    """Ejecuta el GA con manejo de errores"""
    try:
        print("\n" + "="*70)
        print("INICIANDO ALGORITMO GENÉTICO")
        print("="*70)
        
        hof, pop = ejecutar_ga(
            asignaciones, 
            disponibilidad, 
            DIAS, 
            SLOTS,
            W1=W1,
            W2=W2, 
            W3=W3,
            pop_size=100,  # Reducido para evitar crash
            ngen=50,       # Reducido para testing
            cxpb=0.7,
            mutpb=0.3
        )
        
        return hof, pop
        
    except Exception as e:
        print(f"❌ Error durante GA: {e}")
        import traceback
        traceback.print_exc()
        return None, None


def ejecutar_completo(asignaciones, disponibilidad, DIAS, SLOTS, W1=1000, W2=10, W3=100,
                      pop_size=200, ngen=150, validar=True):
    """
    Ejecuta el flujo completo: validación + GA + resultados
    """
    # Paso 1: Validar datos
    if validar:
        print("PASO 1: Validando datos...")
        if not ejecutar_validacion(asignaciones, disponibilidad, DIAS, SLOTS):
            print("\n❌ Validación falló - revisa los errores")
            return None, None
        print("\n✅ Validación exitosa\n")
    
    # Paso 2: Ejecutar GA
    print("PASO 2: Ejecutando algoritmo genético...")
    hof, pop = ejecutar_ga(
        asignaciones, 
        disponibilidad, 
        DIAS, 
        SLOTS,
        W1=W1,
        W2=W2, 
        W3=W3,
        pop_size=pop_size,
        ngen=ngen,
        cxpb=0.7,
        mutpb=0.3
    )
    
    if hof is None or len(hof) == 0:
        print("\n❌ GA no produjo soluciones")
        return None, None
    
    # Paso 3: Mostrar resultados
    print("\n" + "="*70)
    print("PASO 3: Mostrando mejores soluciones")
    print("="*70)
    
    for idx, mejor in enumerate(hof):
        mostrar_solucion(mejor, asignaciones, disponibilidad, idx+1, verbose=(idx==0))
    
    # Paso 4: Crear DataFrame
    mejor_solucion = hof[0]
    df_resultado = crear_dataframe_resultado(mejor_solucion, asignaciones)
    
    print("\n" + "="*70)
    print("MEJOR SOLUCIÓN (DataFrame)")
    print("="*70)
    print(df_resultado.to_string(index=False))
    
    return hof, df_resultado



In [153]:
# TEST
hof, df = ejecutar_completo(
    asignaciones, 
    disponibilidad, 
    DIAS, 
    SLOTS,
    W1=10,      # peso para días usados
    W2=3,       # peso para disponibilidad tipo 2
    W3=0,       # peso para huecos (0 = no penalizar)
    pop_size=200,
    ngen=150,
    validar=False
)

PASO 2: Ejecutando algoritmo genético...
Total de alumnos a programar: 25
Iniciando algoritmo genético...

gen	nevals	avg   	min   	max   
0  	200   	-1e+09	-1e+09	-1e+09
1  	167   	-1e+09	-1e+09	-1e+09
2  	149   	-1e+09	-1e+09	-1e+09
3  	161   	-1e+09	-1e+09	-1e+09
4  	156   	-1e+09	-1e+09	-1e+09
5  	146   	-1e+09	-1e+09	-1e+09
6  	153   	-1e+09	-1e+09	-1e+09
7  	151   	-1e+09	-1e+09	-1e+09
8  	166   	-1e+09	-1e+09	-1e+09
9  	160   	-1e+09	-1e+09	-1e+09
10 	159   	-1e+09	-1e+09	-1e+09
11 	151   	-1e+09	-1e+09	-1e+09
12 	171   	-1e+09	-1e+09	-1e+09
13 	154   	-1e+09	-1e+09	-1e+09
14 	161   	-1e+09	-1e+09	-1e+09
15 	161   	-1e+09	-1e+09	-1e+09
16 	143   	-1e+09	-1e+09	-1e+09
17 	169   	-1e+09	-1e+09	-1e+09
18 	159   	-1e+09	-1e+09	-1e+09
19 	164   	-1e+09	-1e+09	-1e+09
20 	157   	-1e+09	-1e+09	-1e+09
21 	159   	-1e+09	-1e+09	-1e+09
22 	154   	-1e+09	-1e+09	-1e+09
23 	166   	-1e+09	-1e+09	-1e+09
24 	167   	-1e+09	-1e+09	-1e+09
25 	166   	-1e+09	-1e+09	-1e+09
26 	154   	-1e+09	-1e+09	-1e+